In [ ]:
from astropy.io import fits
from astropy.stats import sigma_clip
import numpy as np
from scipy import ndimage

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [ ]:
fits_path = '../casapy-simulation-scripts/simulation_output/vla.image.fits'

Load the data. A mask can be applied if necessary - this may be useful e.g. for excluding the region around a bright source, to avoid false detections due to sidelobes.

In [ ]:
hdu0 = fits.open(fits_path)[0]
imgdata = hdu0.data.squeeze()
# imgdata = np.ma.MaskedArray(imgdata, mask=np.zeros_like(imgdata))
# imgdata.mask[900:1100,900:1100] = True
# imgdata.mask.any()

In [ ]:
# Plot pixels in cartesian ordering:
plt.rcParams['image.origin'] = 'lower'
# Make plots bigger
plt.rcParams['figure.figsize'] = 10, 10

In [ ]:
imgmax = np.max(imgdata)
plt.imshow(imgdata,vmax=imgmax*0.5)
plt.colorbar()

Crudely estimate the background level and RMS:

In [ ]:
clipped = sigma_clip(imgdata)
rms = np.ma.std(clipped)
bg_estimate = np.ma.median(imgdata)

We use two thresholds when identifying our source 'islands' (connected pixel regions). The high threshold is our detection level, and should be set high enough to avoid false detections due to noise spikes. The lower threshold expands each island, such that it is more likely to contain enough pixels to reasonably fit a Gaussian profile (otherwise the island may consist of only a single pixel over the detection threshold).

Note that this thresholding approach may result in multi-peaked regions (e.g. two distinct but adjacent sources) being assigned to a single island / label. This can be tackled with 'deblending' algorithms if desired, but is not covered in this notebook.

In [ ]:
analysis_threshold = bg_estimate + 3*rms
detection_threshold = bg_estimate + 6*rms

In [ ]:
analysis_pix = imgdata > analysis_threshold
high_pix = imgdata > detection_threshold

In [ ]:
plt.imshow(high_pix)

In [ ]:
plt.imshow(analysis_pix)

In [ ]:
labeldata, n_labels = ndimage.label(analysis_pix)

Copy the mask from the input dataset, if it has one (ndimage.label doesn't account for masks):

In [ ]:
labeldata = np.ma.MaskedArray(labeldata, mask=np.ma.getmask(imgdata))

Check the label of our bright central source:

In [ ]:
labeldata[1010,1010]

In [ ]:
plt.imshow(labeldata)

In [ ]:
def get_label_mask(labeldata, label_num):
    return ~(labeldata==label_num)

In [ ]:
def get_island_data(imgdata,labeldata,label_num):
    return np.ma.MaskedArray(imgdata, mask=( get_label_mask(labeldata, label_num) ))

In [ ]:
central_source_island = get_island_data(imgdata,labeldata, label_num=29)
# unmasked_island = get_island_data(imgdata.data,labeldata, label_num=29)

In [ ]:
plt.imshow(central_source_island)

Now we loop over all 'analysis threshold' level islands and see which ones peak above the detection level.

In [ ]:
clear_detection_labels = []
for i in range(n_labels):
    if (get_island_data(imgdata,labeldata,label_num=i)>detection_threshold).any():
        clear_detection_labels.append(i)

In [ ]:
clear_detection_labels

As a final check, let's plot the islands which appear to be 'clear detections':

In [ ]:
# Mask all pixels to start with:
combined_mask = np.ones_like(imgdata)
for i in clear_detection_labels:
    # Combine with each island mask to open a hole for each source.
    combined_mask = np.logical_and(combined_mask, get_label_mask(labeldata,i))
all_clear_detections_data = np.ma.MaskedArray(imgdata, mask=combined_mask)

In [ ]:
plt.imshow(all_clear_detections_data)

Get the peak pixel for each island. The function ``np.ma.argmax`` returns the index of the peak pixel in the flattened data, so we massage it back into (y,x) index form:

In [ ]:
def get_peak_pixel_index(island_data):
    return np.unravel_index(np.ma.argmax(island), imgdata.shape)

In [ ]:
island_num = clear_detection_labels[-1]
island = get_island_data(imgdata, labeldata, island_num)
peak_pixel_index = get_peak_pixel_index(island)
peak_pixel_index

In [ ]:
peak_pixel_value = island[peak_pixel_index]
peak_pixel_value 

In [ ]:
plt.imshow(island)
y, x = peak_pixel_index
plt.scatter(x,y, marker='*', s=200, c='y',)
plt.xlim(x-50,x+50)
plt.ylim(y-50,y+50)